## Efficiently finding frequent itemsets with FP-growth

- Pros: Usually faster than Apriori.
- Cons: Difficult to implement; certain datasets degrade the performance.
- Works with: Nominal values.

In [1]:
import re

### Implement FPGrowth

In [2]:
# FP-tree class definition
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}

    def inc(self, numOccur):
        self.count += numOccur

    def disp(self, ind=1):
        print('  ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

In [3]:
# FP-tree creation code
def createTree(dataSet, minSup=1):
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # Remove items not meeting min support
    for k in list(headerTable.keys()):
        if headerTable[k] < minSup:
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    # If no items meet min support, exit
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        localD = {}
        # Sort transactions by global frequency
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(),
                                                 key=lambda p: p[1],
                                                 reverse=True)]
            # Populate tree with ordered freq itemset
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable


def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] is None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        # Recursively call updateTree on remaining items
        updateTree(items[1:], inTree.children[items[0]], headerTable, count)


def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink is not None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [4]:
# A function to find all paths ending with a given item
def ascendTree(leafNode, prefixPath):
    # Recursively ascend the tree
    if leafNode.parent is not None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)


def findPrefixPath(treeNode):
    condPats = {}
    while treeNode is not None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [5]:
# The mineTree function recursively finds frequent itemsets
# Get conditional pattern bases from the FP-tree.
# From the conditional pattern base, construct a conditional FP-tree.
# Recursively repeat steps 1 and 2 on until the tree contains a single item.
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # Start from bottom of header table
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(headerTable[basePat][1])
        # Construct cond. FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        if myHead is not None:
            # Mine cond. FP-tree
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

### Experiment 1: Toy dataset

In [6]:
# Simple dataset and data wrapper
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [7]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        if frozenset(trans) in retDict:
            retDict[frozenset(trans)] += 1
        else:
            retDict[frozenset(trans)] = 1
    return retDict

In [8]:
simpDat = loadSimpDat()

In [9]:
initSet = createInitSet(simpDat)

In [10]:
myFPtree, myHeaderTab = createTree(initSet, 2)

In [11]:
myFPtree.disp()

   Null Set   1
     z   5
       r   1
         p   1
       x   3
         y   2
           t   2
             s   2
               q   1
         r   1
           y   1
             t   1
               q   1
                 p   1
     x   1
       r   1
         s   1


In [12]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)
print(freqItems)

[{'p'}, {'q'}, {'r'}, {'y'}, {'x', 'y'}, {'z', 'y'}, {'x', 'z', 'y'}, {'t'}, {'x', 't'}, {'z', 't'}, {'x', 'z', 't'}, {'y', 't'}, {'x', 'y', 't'}, {'z', 'y', 't'}, {'x', 'z', 'y', 't'}, {'s'}, {'x', 's'}, {'x'}, {'x', 'z'}, {'z'}]


### Experiment 2: Twitter dataset

In [13]:
# Text parsing and glue code
# twitter API difficult to apply
# datasets based on http://help.sentiment140.com/home
# select tweets which contain word internet
def textParse(bigString):
    urlsRemoved = re.sub('(http:[/][/]|www.)([a-z]|[A-Z]|[0-9]|[/.]|[~])*',
                         '', bigString)
    listOfTokens = re.split(r'\W+', urlsRemoved)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]


def mineTweets(minSup=250):
    parsedList = []
    input_file = open("twitter.txt")
    lines = input_file.readlines()
    input_file.close()
    for line in lines:
        parsedList.append(textParse(line))
    initSet = createInitSet(parsedList)
    myFPtree, myHeaderTab = createTree(initSet, minSup)
    myFreqList = []
    mineTree(myFPtree, myHeaderTab, minSup, set([]), myFreqList)
    return myFreqList

In [14]:
mineTweets()

[{'home'},
 {'from'},
 {'slow'},
 {'internet', 'slow'},
 {'work'},
 {'internet', 'work'},
 {'down'},
 {'down', 'internet'},
 {'was'},
 {'internet', 'was'},
 {'back'},
 {'back', 'internet'},
 {'this'},
 {'internet', 'this'},
 {'all'},
 {'all', 'internet'},
 {'out'},
 {'internet', 'out'},
 {'get'},
 {'get', 'internet'},
 {'connection'},
 {'connection', 'internet'},
 {'just'},
 {'internet', 'just'},
 {'with'},
 {'internet', 'with'},
 {'not'},
 {'internet', 'not'},
 {'can'},
 {'can', 'internet'},
 {'now'},
 {'internet', 'now'},
 {'that'},
 {'that', 'the'},
 {'internet', 'that'},
 {'you'},
 {'the', 'you'},
 {'internet', 'you'},
 {'but'},
 {'but', 'the'},
 {'but', 'internet', 'the'},
 {'but', 'internet'},
 {'have'},
 {'have', 'the'},
 {'have', 'internet', 'the'},
 {'have', 'internet'},
 {'for'},
 {'for', 'the'},
 {'for', 'internet', 'the'},
 {'for', 'internet'},
 {'and'},
 {'and', 'the'},
 {'and', 'internet', 'the'},
 {'and', 'internet'},
 {'the'},
 {'internet', 'the'},
 {'internet'}]

### Experiment 3: Clickstream dataset

In [15]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]
initSet = createInitSet(parsedDat)
myFPtree, myHeaderTab = createTree(initSet, 100000)

In [16]:
myFreqList = []
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)
print(myFreqList)

[{'1'}, {'6', '1'}, {'11'}, {'3', '11'}, {'3', '6', '11'}, {'6', '11'}, {'3'}, {'6', '3'}, {'6'}]
